In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

# スコア評価用ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support

# 特徴量選択用
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

# モデルの読み込み、保存
from sklearn.externals import joblib

# エラー表示の抑制
import warnings

D:\Program\Anaconda3\envs\sklearn1.9.1\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [3]:
# テキストデータの読み込み
wikiData = pd.read_csv("csv/train_cleaned.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー Dole Food Comp...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場 みなみ じょう 栃木県 那須高原 本拠 酪...,1
2,岩瀬牧場 岩瀬牧場 ぼくじょう 日本 福島県 岩瀬 郡 鏡石町 牧場 鏡石...,1
3,くら ぎ GI Co Ltd 三重県 中心 農業 店舗 展開 会社 本...,1
4,ハッピーネモファーム 株式会社 ハッピーネモファーム 北海道 浦河 郡 浦河町 ...,1


In [4]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [5]:
# トレーニングデータ、テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in ['名詞'] :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [8]:
# TF-IDFへ変換
vectorizer = joblib.load('model/Tfidf_vectorizer.pkl')
X_train = vectorizer.transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

In [9]:
# Light GBM
import lightgbm as lgb

clf = lgb.LGBMClassifier(objective='multiclass', num_class=33, random_state=42)

clf.fit(X_train, y_train)
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)

print("Training score：" + str(clf.score(X_train, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf.score(X_test, y_test)))

Training score：0.998254037538
Cross-Validation score：0.70973098604
Test score：0.731994762113


In [10]:
# 学習結果の保存
joblib.dump(clf, 'model/lgbm_Tfidf.pkl')

['model/lgbm_Tfidf.pkl']